<h1>SpaceX  Falcon 9 first stage Landing Prediction</h1>

---

# 3.1 Exploratory Data Analysis with SQL

### Objectives
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to Understand the Spacex DataSet

Prevoiusly refined DataSet: <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [ ]:
# !pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# # Ensure we don't load_ext with sqlalchemy>=1.4 (incompatible)
# !pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24

# !pip install ipython-sql

## DataBase Connection

Load the SQL magic extension and establish a connection with the DB.

```
%sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL

```




In [ ]:
import csv, sqlite3
import pandas as pd
import ibm_db

%load_ext sql

In [ ]:
# CODE CELL INCLUDING RESERVED CREDENTIALS

In [ ]:
# Create database connection
dsn = ( "DRIVER={0};"
        "DATABASE={1};"
        "HOSTNAME={2};"
        "PORT={3};"
        "PROTOCOL={4};"
        "UID={5};"
        "PWD={6};"
        "SECURITY={7};"
      ).format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd, dsn_security)

try:
  conn = ibm_db.connect(dsn, "", "")
  print ("Done")
except: print ("Unable to connect: ", ibm_db.conn_errormsg() )

Done


In [ ]:
# CODE CELL INCLUDING RESERVED CREDENTIALS

Connected to database


Alternatively, one can create a temporary session run DataBase, using $\verb|sqlite|$, provided the CSV DataSet <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">link</a>.



```
con = sqlite3.connect("my_data.db")
cur = con.cursor()

%sql sqlite:///my_data.db

df = pd.read_csv("link")
df.to_sql("SpaceX_Table", con, if_exists='replace', index=False, method="multi")
```



In [ ]:
%sql select count(*) as total from spacex;

 * ibm_db_sa://try34089:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


total
999


## Data Exploration


#### 1.
##### Display the names of the unique launch sites  in the space mission

In [ ]:
%%sql

select distinct launch_site as Launch_Sites
from spacex
where launch_site is not null;

 * ibm_db_sa://try34089:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


launch_sites
CCAFS LC-40
CCAFS SLC-40
KSC LC-39A
VAFB SLC-4E



#### 2.
##### Display 5 records where launch sites begin with the string 'CCA'



In [ ]:
%%sql

select *
from spacex
where launch_site like 'CCA%'
limit 5;

 * ibm_db_sa://try34089:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


DATE,time_utc,booster_version,launch_site,payload,payload_mass,orbit,customer,mission_outcome,landing_outcome
2010-04-06,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-08-12,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-08-10,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-01-03,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


#### 3.
##### Display the total payload mass carried by boosters launched by NASA (CRS)






In [ ]:
%%sql

select SUM(payload_mass) as total_payload
from spacex
where customer like '%CRS%';

 * ibm_db_sa://try34089:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


total_payload
48213


#### 4.
##### Display average payload mass carried by booster version F9 v1.1



In [ ]:
%%sql

select AVG(payload_mass) as avg_payload
from spacex
where booster_version = 'F9 v1.1';

 * ibm_db_sa://try34089:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


avg_payload
2928


#### 5.  
##### List the date when the first succesful landing outcome in ground pad was acheived.


In [ ]:
%%sql

select date, time_utc, landing_outcome
from spacex
where landing_outcome = 'Success (ground pad)'
order by date asc
limit 1;

 * ibm_db_sa://try34089:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


DATE,time_utc,landing_outcome
2015-12-22,1:29:00,Success (ground pad)


#### 6.
##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000



In [ ]:
%%sql

select booster_version, payload_mass
from spacex
where landing_outcome in ( 'Success (drone ship)' )
      and payload_mass between 4000 and 6000;

 * ibm_db_sa://try34089:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


booster_version,payload_mass
F9 FT B1022,4696
F9 FT B1026,4600
F9 FT B1021.2,5300
F9 FT B1031.2,5200


#### 7.
##### List the total number of successful and failure mission outcomes







In [ ]:
%%sql

select
  case
    when mission_outcome like 'Success%' then 'Success'
    else 'Failure'
  end as outcome,
  count(*) as total

from spacex
where mission_outcome is not null
group by
  case
    when mission_outcome like 'Success%' then 'Success'
    else 'Failure'
  end;

 * ibm_db_sa://try34089:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


outcome,total
Failure,1
Success,100


#### 8.
##### List the   names of the booster_versions which have carried the maximum payload mass.



In [ ]:
%%sql

select distinct booster_version, payload_mass
from spacex
where payload_mass = (select max(payload_mass) from spacex)
order by 1;

 * ibm_db_sa://try34089:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


booster_version,payload_mass
F9 B5 B1048.4,15600
F9 B5 B1048.5,15600
F9 B5 B1049.4,15600
F9 B5 B1049.5,15600
F9 B5 B1049.7,15600
F9 B5 B1051.3,15600
F9 B5 B1051.4,15600
F9 B5 B1051.6,15600
F9 B5 B1056.4,15600
F9 B5 B1058.3,15600


#### 9.
##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.



In [ ]:
%%sql
-- STRFTIME('%m', date) not available in IBM db2
select month(date) as month, booster_version, launch_site, landing_outcome
from spacex
where year(date) in ('2015')
      and landing_outcome in ('Failure (drone ship)')
order by 1;

 * ibm_db_sa://try34089:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


MONTH,booster_version,launch_site,landing_outcome
4,F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)
10,F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)


#### 10.  
##### Rank the count of landing outcomes between the date 2010-06-04 and 2017-03-20, in descending order.







In [ ]:
%%sql

select landing_outcome, count(*) as total
from spacex
where date between '2010-06-04' and '2017-03-20'
group by landing_outcome
order by 2 desc;

 * ibm_db_sa://try34089:***@98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud:30875/BLUDB
Done.


landing_outcome,total
No attempt,10
Failure (drone ship),5
Success (drone ship),5
Success (ground pad),5
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),1
Precluded (drone ship),1
